In [1]:
import serial
import threading
import time
import datetime
import csv
from collections import deque
import statistics
import nest_asyncio

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Allow asyncio inside Jupyter
nest_asyncio.apply()

# --- Serial Setup ---
SERIAL_PORT = "COM10"   # change if needed
BAUD_RATE = 9600
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

# --- Data Buffers ---
MAX_POINTS = 200
data_history = {
    "timestamp": deque(maxlen=MAX_POINTS),
    "waterLevel": deque(maxlen=MAX_POINTS),
    "drinkCount": deque(maxlen=MAX_POINTS),
    "status": deque(maxlen=MAX_POINTS),
    "led": deque(maxlen=MAX_POINTS),
    "temperature": deque(maxlen=MAX_POINTS),
    "humidity": deque(maxlen=MAX_POINTS),
}

latest_data = {
    "waterLevel": 0,
    "drinkCount": 0,
    "status": "UNKNOWN",
    "led": "OFF",
    "temperature": 0,
    "humidity": 0,
}

# --- Files ---
date_str = datetime.datetime.now().strftime("%Y-%m-%d")
detailed_file = f"detailed_log_{date_str}.csv"
summary_file = f"hydration_summary_{date_str}.csv"
DAILY_GOAL = 2000  # ml

# Init detailed file
with open(detailed_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Timestamp", "WaterLevel_ml", "DrinkCount", "Status", "LED", "Temp_C", "Hum_%"])

# Init summary file
with open(summary_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Date", "Total_Consumed_ml", "Daily_Goal_ml", "Goal_Achievement_%", 
                     "Drink_Count", "Avg_Temp", "Avg_Humidity"])

# --- Background Serial Reader ---
def read_serial():
    while True:
        try:
            line = ser.readline().decode("utf-8").strip()
            if line.startswith("WaterLevel"):
                parts = dict(item.split(":") for item in line.split(","))
                now = datetime.datetime.now()

                latest_data["waterLevel"] = float(parts.get("WaterLevel", 0))
                latest_data["drinkCount"] = int(parts.get("DrinkCount", 0))
                latest_data["status"] = parts.get("Status", "UNKNOWN")
                latest_data["led"] = parts.get("LED", "OFF")
                latest_data["temperature"] = float(parts.get("Temp", 0))
                latest_data["humidity"] = float(parts.get("Hum", 0))

                # Append to history
                data_history["timestamp"].append(now)
                for key in latest_data:
                    data_history[key].append(latest_data[key])

                # Write to detailed log (every second)
                with open(detailed_file, "a", newline="", encoding="utf-8") as f:
                    writer = csv.writer(f)
                    writer.writerow([
                        now.strftime("%Y-%m-%d %H:%M:%S"),
                        latest_data["waterLevel"],
                        latest_data["drinkCount"],
                        latest_data["status"],
                        latest_data["led"],
                        latest_data["temperature"],
                        latest_data["humidity"]
                    ])
        except Exception as e:
            print("Serial error:", e)
            time.sleep(1)

serial_thread = threading.Thread(target=read_serial, daemon=True)
serial_thread.start()

# --- Summary Writer (every 30 sec) ---
def write_summary():
    while True:
        if data_history["timestamp"]:
            total_consumed = latest_data["drinkCount"] * 200  # 200ml per drink
            efficiency = round((total_consumed / DAILY_GOAL) * 100, 2)
            avg_temp = round(statistics.mean(data_history["temperature"]), 1)
            avg_hum = round(statistics.mean(data_history["humidity"]), 1)

            with open(summary_file, "a", newline="", encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow([
                    date_str,
                    total_consumed,
                    DAILY_GOAL,
                    efficiency,
                    latest_data["drinkCount"],
                    avg_temp,
                    avg_hum
                ])
        time.sleep(30)

summary_thread = threading.Thread(target=write_summary, daemon=True)
summary_thread.start()

# --- Dash Dashboard ---
app = dash.Dash(__name__, title="HydroSync Dashboard")

app.layout = html.Div([
    html.H1("HydroSync: Smart Water Intake Tracker", style={"textAlign": "center"}),

    html.Div([
        dcc.Graph(id="water-gauge", style={"width": "45%", "display": "inline-block"}),
        dcc.Graph(id="drink-count", style={"width": "45%", "display": "inline-block"}),
    ]),

    html.Div([
        html.Div(id="status-text", style={
            "fontSize": "22px", "color": "blue", "marginTop": "20px"
        }),
    ], style={"textAlign": "center"}),

    html.Div([
        dcc.Graph(id="temp-humidity-plot", style={"width": "48%", "display": "inline-block"}),
        dcc.Graph(id="waterlevel-plot", style={"width": "48%", "display": "inline-block"}),
    ]),

    dcc.Interval(id="interval", interval=1000, n_intervals=0)
])

@app.callback(
    Output("water-gauge", "figure"),
    Output("drink-count", "figure"),
    Output("status-text", "children"),
    Output("temp-humidity-plot", "figure"),
    Output("waterlevel-plot", "figure"),
    Input("interval", "n_intervals")
)
def update_dashboard(n):
    wl = latest_data["waterLevel"]
    dc = latest_data["drinkCount"]
    status = latest_data["status"]
    led = latest_data["led"]

    # Gauge
    gauge = go.Figure(go.Indicator(
        mode="gauge+number",
        value=wl,
        title={"text": "Water Level (ml)"},
        gauge={"axis": {"range": [0, 665]}, "bar": {"color": "blue"}}
    ))

    # Drink counter
    count_fig = go.Figure(go.Indicator(
        mode="number",
        value=dc,
        title={"text": "Drinks Today"}
    ))

    # Status text
    status_msg = f"Cup Status: {status} | LED: {led}"

    # Environment plot
    times = list(data_history["timestamp"])
    temp_fig = go.Figure()
    if times:
        temp_fig.add_trace(go.Scatter(x=times, y=list(data_history["temperature"]),
                                      mode="lines", name="Temperature (°C)", line=dict(color="red")))
        temp_fig.add_trace(go.Scatter(x=times, y=list(data_history["humidity"]),
                                      mode="lines", name="Humidity (%)", line=dict(color="blue")))
    temp_fig.update_layout(title="Environment", xaxis_title="Time")

    # Water level trend
    wl_fig = go.Figure()
    if times:
        wl_fig.add_trace(go.Scatter(x=times, y=list(data_history["waterLevel"]),
                                    mode="lines+markers", name="Water Level"))
    wl_fig.update_layout(title="Water Level Over Time", xaxis_title="Time", yaxis_title="ml")

    return gauge, count_fig, status_msg, temp_fig, wl_fig

# --- Run Dashboard ---
if __name__ == "__main__":
    print("Starting HydroSync dashboard... Go to the URL below.")
    app.run(debug=True, jupyter_mode="tab")


Starting HydroSync dashboard... Go to the URL below.
Dash app running on http://127.0.0.1:8050/


C:\Users\letan\anaconda3\envs\clean_env\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


<IPython.core.display.Javascript object>